In [ ]:
import csv
from io import StringIO

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pandas import Series


Reading the raw dataset. Some rows (e.g. line 1010) contain complex nested quotes and commas, which requires modifying each line before passing it to `pandas.read_csv()`. The outer quotes get replaced by an arbitrary (`\u2603`, the snowman) character, which is then specified as the quote character used for parsing.

In [ ]:
csv.field_size_limit(4 * 131072)  # 4 times the default

with open(
    "../raw/webis_bias_flipper_18.csv", mode="r", encoding="utf-8"
) as file:
    modified_lines = (
        "\u2603".join(
            line.replace('","', "\u2603,\u2603")
            .replace('"', "\u2603", 1)
            .rsplit('"', 1)
        )
        for line in file
    )
    df = pd.read_csv(
        StringIO("".join(modified_lines)),
        quotechar="\u2603",
        encoding="utf-8",
        engine="python",
    )
df


Comparison of the body column (from AllSides) versus the original body column (from the news portals). The text volume of the original columns is much higher, probably because AllSides cuts the length. This means the title and body columns can be dropped and the original ones will be used.

In [ ]:
pd.Series(
    [df["original_body"].str.len().sum(), df["body"].str.len().sum()],
    index=["original body", "body"],
).plot.pie(autopct="%1.1f%%")


Dropping useless columns.

In [ ]:
df = df.drop(columns=["story_id", "title", "body", "source"])


Renaming and reordering columns.

In [ ]:
df = df.rename(
    columns={
        "original_title": "title",
        "original_body": "body",
        "bias": "leaning",
    }
)
df = df[["title", "body", "leaning"]].copy()


Categorizing the leaning label column.

In [ ]:
df["leaning"] = df["leaning"].astype("category")
df["leaning"] = df["leaning"].cat.rename_categories(
    {
        "From the Left": "left",
        "From the Center": "center",
        "From the Right": "right",
    }
)


Adding the politicalness label column.

In [ ]:
df["politicalness"] = pd.Categorical(["political"] * len(df))


Stripping the titles and the bodies.

In [ ]:
df["title"] = (df["title"].str.strip()).replace("", np.nan)
df["body"] = (df["body"].str.strip()).replace("", np.nan)


Dropping rows missing either the leaning or both the title and the body.

In [ ]:
df = df.dropna(subset=["leaning"])
df = df.dropna(subset=["title", "body"], how="all")


Printing body duplicates.

In [ ]:
df[df["body"].duplicated(keep=False)]


Dropping the body duplicates.

In [ ]:
df = df.drop_duplicates(subset="body")


Inspecting the title and the body length.

In [ ]:
df["title_length"] = df["title"].fillna("").str.len()
df["title_word_count"] = df["title"].fillna("").str.split().str.len()
df["body_length"] = df["body"].fillna("").str.len()
df["body_word_count"] = df["body"].fillna("").str.split().str.len()
df = df.sort_values(by="body_length")
df.head()


After the inspection, rows with bodies shorter than 15 words seem to contain no political value. Removing them.

In [ ]:
body_word_count_lower_bound = 15
df = df[df["body_word_count"] >= body_word_count_lower_bound]


In [ ]:
df.to_parquet("../preprocessed/webis_bias_flipper_18.parquet")
